# PROCESO ET (EXTRACCION Y TRANSFORMACION):
Recurrimos a una tabla fuente de una pagina, para extraer informacion y luego hacer transformaciones usando la biblioteca "re" y "datetime"

## PRIMERA ETAPA: Extraccion de fuente con "BeatifulSoup"
Extramos las fechas en cual son feriados en EEUU, ademas de la descripcion de cada una de ellas. Las mismas se encuentran en la pag.

In [ ]:
import requests

In [ ]:
x=requests.get("https://www.opm.gov/policy-data-oversight/pay-leave/federal-holidays/#url=2023")
type(x)

requests.models.Response

In [ ]:
x.text

'\r\n\r\n\r\n\r\n<!DOCTYPE html>\r\n\r\n<html   xmlns="http://www.w3.org/1999/xhtml"      xmlns:og="http://opengraphprotocol.org/schema/"      xmlns:fb="http://www.facebook.com/2008/fbml"      lang="en"      xml:lang="en"    >\r\n<head>\r\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge" />\r\n\t<link rel="shortcut icon" href="/favicon.ico" />\r\n\r\n\t\r\n\t\r\n\t<title>Federal Holidays</title>\r\n\t<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\r\n\t<meta name="description" content="Welcome to opm.gov" />\r\n\t<meta name="keywords" content="OPM,Office of Personnel Management,opm.gov" />\r\n\t<meta name="Expires" content="" />\r\n\t<meta name="TemplateVersion" content="3.0" />\r\n\t<meta name="viewport" content="width=device-width" />\r\n\r\n\t\r\n    \r\n\r\n\r\n\t<meta property="fb:admins" content="568256384" />\r\n\t<meta property="fb:app_id" content="121223957945585" />\r\n\t<meta property="og:type" content="government" />\r\n\t<meta property="og:site_

In [ ]:
from bs4 import BeautifulSoup as bs

In [ ]:
sopa=bs(x.content)

In [ ]:
# Extraemos tabla de interes
table = sopa.find("table", class_="DataTable HolidayTable")

In [ ]:
# Exploramos si corresponde con la informacion de la tabla
table.text

'\n2023 Holiday Schedule\n\n\nDate\nHoliday\n\n\n\n\nMonday, January 02 *\nNew Year’s Day\n\n\nMonday, January 16 \nBirthday of Martin Luther King, Jr.\n\n\nMonday, February 20 **\nWashington’s Birthday\n\n\nMonday, May 29 \nMemorial Day\n\n\nMonday, June 19 \nJuneteenth National Independence Day\n\n\nTuesday, July 04 \nIndependence Day\n\n\nMonday, September 04 \nLabor Day\n\n\nMonday, October 09 \nColumbus Day\n\n\nFriday, November 10 *\nVeterans Day\n\n\nThursday, November 23 \nThanksgiving Day\n\n\nMonday, December 25 \nChristmas Day\n\n\n'

In [ ]:
# Extramos todas las filas de la tabla
rows= table.find_all("tr")

In [ ]:
# Examinamos nombre de variables:
nombre_variables= table.find_all("tr")[0]
print(nombre_variables)

<tr>
<th scope="col">Date</th>
<th scope="col">Holiday</th>
</tr>


In [ ]:
# Examinamos celda ubicada en: Fila=1,Columna=0.
F1_C0= rows[1].find_all("td")[0]
print(F1_C0)

<td>Monday, January 02 <span class="Footnote">*</span></td>


In [ ]:
# Examinamos celda ubicada en: Fila=1,Columna=1.
F1_C1= rows[1].find_all("td")[1]
print(F1_C1)


<td>New Year’s Day</td>


In [ ]:
Date=[]
Description=[]
for x in rows [1:13]:
  Date.append(x.find_all("td")[0].text)
  Description.append(x.find_all("td")[1].text)

In [ ]:
Date

['Monday, January 02 *',
 'Monday, January 16 ',
 'Monday, February 20 **',
 'Monday, May 29 ',
 'Monday, June 19 ',
 'Tuesday, July 04 ',
 'Monday, September 04 ',
 'Monday, October 09 ',
 'Friday, November 10 *',
 'Thursday, November 23 ',
 'Monday, December 25 ']

In [ ]:
data = {'Date': Date,'Description': Description}

In [ ]:
import pandas as pd
df= pd.DataFrame(data)

In [ ]:
df.head(11)

,Date,Description
0,"Monday, January 02 *",New Year’s Day
1,"Monday, January 16","Birthday of Martin Luther King, Jr."
2,"Monday, February 20 **",Washington’s Birthday
3,"Monday, May 29",Memorial Day
4,"Monday, June 19",Juneteenth National Independence Day
5,"Tuesday, July 04",Independence Day
6,"Monday, September 04",Labor Day
7,"Monday, October 09",Columbus Day
8,"Friday, November 10 *",Veterans Day
9,"Thursday, November 23",Thanksgiving Day


## SEGUNDA ETAPA: Procesamiento con "re"
Acondicionamos la cadenas de caracteres en la columna Date, para posteriormente poder hacer una transformacion con la biblioteca Time, mas prolija.

In [ ]:
df

,Date,Description
0,"Monday, January 02 *",New Year’s Day
1,"Monday, January 16","Birthday of Martin Luther King, Jr."
2,"Monday, February 20 **",Washington’s Birthday
3,"Monday, May 29",Memorial Day
4,"Monday, June 19",Juneteenth National Independence Day
5,"Tuesday, July 04",Independence Day
6,"Monday, September 04",Labor Day
7,"Monday, October 09",Columbus Day
8,"Friday, November 10 *",Veterans Day
9,"Thursday, November 23",Thanksgiving Day


In [ ]:
import re

In [ ]:
# Creamos patron, para identificar apostrofes al final de las cadena de texto en la columna "Date"
patron=r'\s?\W\W$'

In [ ]:
texto ="Monday, February 20 *"

In [ ]:
# Hacemos una prueba para ver si elimina
cadena_sin_apostrofe= re.sub(patron,'',texto)

In [ ]:
print(cadena_sin_apostrofe)

Monday, February 20


In [ ]:
# Primer filtrado: Eliminacion de apostrofes al final de la cadena
import re
def eliminar_apostrofes(cadena):
    return re.sub(r'\s*\*$', '', cadena) #Primera expresion regular
df['Date'] = df['Date'].apply(eliminar_apostrofes) #Usamos apply de pandas
print(df)

                      Date                           Description
0       Monday, January 02                        New Year’s Day
1      Monday, January 16    Birthday of Martin Luther King, Jr.
2    Monday, February 20 *                 Washington’s Birthday
3          Monday, May 29                           Memorial Day
4         Monday, June 19   Juneteenth National Independence Day
5        Tuesday, July 04                       Independence Day
6    Monday, September 04                              Labor Day
7      Monday, October 09                           Columbus Day
8      Friday, November 10                          Veterans Day
9   Thursday, November 23                       Thanksgiving Day
10    Monday, December 25                          Christmas Day


In [ ]:
# Segundo filtrado: Eliminacion de los apostrofes restantes
def eliminar_apostrofes(cadena):
    return re.sub(r'\s*\*$', '', cadena)
df['Date'] = df['Date'].apply(eliminar_apostrofes) #Usamos apply de pandas
print(df)

                      Date                           Description
0       Monday, January 02                        New Year’s Day
1      Monday, January 16    Birthday of Martin Luther King, Jr.
2      Monday, February 20                 Washington’s Birthday
3          Monday, May 29                           Memorial Day
4         Monday, June 19   Juneteenth National Independence Day
5        Tuesday, July 04                       Independence Day
6    Monday, September 04                              Labor Day
7      Monday, October 09                           Columbus Day
8      Friday, November 10                          Veterans Day
9   Thursday, November 23                       Thanksgiving Day
10    Monday, December 25                          Christmas Day


In [ ]:
# Tercer filtrado: Eliminamos los espaciso dentro de las cadenas de caracteres.
def eliminar_espacios(cadena):
    return re.sub(r'\s+', '', cadena)
df['Date'] = df['Date'].apply(eliminar_espacios) #Forma simplificada, usando apply de pandas
print(df)

                   Date                           Description
0      Monday,January02                        New Year’s Day
1      Monday,January16   Birthday of Martin Luther King, Jr.
2     Monday,February20                 Washington’s Birthday
3          Monday,May29                          Memorial Day
4         Monday,June19  Juneteenth National Independence Day
5        Tuesday,July04                      Independence Day
6    Monday,September04                             Labor Day
7      Monday,October09                          Columbus Day
8     Friday,November10                          Veterans Day
9   Thursday,November23                      Thanksgiving Day
10    Monday,December25                         Christmas Day


In [ ]:
# Agregamos str ",2023" a cada fila de la columna Date
df['Date'] = df['Date'] + ',2023'

In [ ]:
df["Date"]

0        Monday,January02,2023
1        Monday,January16,2023
2       Monday,February20,2023
3            Monday,May29,2023
4           Monday,June19,2023
5          Tuesday,July04,2023
6      Monday,September04,2023
7        Monday,October09,2023
8       Friday,November10,2023
9     Thursday,November23,2023
10      Monday,December25,2023
Name: Date, dtype: object

## TERCERA ETAPA: Procesamiento con "datetime"
Realizamos la transformacion de las fechas "str" de la columna Date , a time.

In [ ]:
df.head(11)

,Date,Description
0,2023-01-02,New Year’s Day
1,2023-01-16,"Birthday of Martin Luther King, Jr."
2,2023-02-20,Washington’s Birthday
3,2023-05-29,Memorial Day
4,2023-06-19,Juneteenth National Independence Day
5,2023-07-04,Independence Day
6,2023-09-04,Labor Day
7,2023-10-09,Columbus Day
8,2023-11-10,Veterans Day
9,2023-11-23,Thanksgiving Day


In [ ]:
# Transformamos la columna date.
from datetime import datetime
formato = "%A,%B%d,%Y" #Identificamos el formate de la fecha texto correspondiente a Date
def transformacion_time(cadena): #Creamos funcion para aplicarla a cada elemento de la columna.
  return datetime.strptime(cadena, formato)
df['Date'] = df['Date'].apply(transformacion_time)
print(df)

         Date                           Description
0  2023-01-02                        New Year’s Day
1  2023-01-16   Birthday of Martin Luther King, Jr.
2  2023-02-20                 Washington’s Birthday
3  2023-05-29                          Memorial Day
4  2023-06-19  Juneteenth National Independence Day
5  2023-07-04                      Independence Day
6  2023-09-04                             Labor Day
7  2023-10-09                          Columbus Day
8  2023-11-10                          Veterans Day
9  2023-11-23                      Thanksgiving Day
10 2023-12-25                         Christmas Day
